https://duckdb.org/docs/guides/sql_editors/dbeaver 


In [3]:
import duckdb

In [4]:
r1 = duckdb.sql('SELECT 42 AS i')
r1

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘

In [5]:
duckdb.sql("SELECT i*100 AS k FROM r1")

┌───────┐
│   k   │
│ int32 │
├───────┤
│  4200 │
└───────┘

In [6]:
duckdb.sql("SELECT i*100 FROM r1")

┌───────────┐
│ (i * 100) │
│   int32   │
├───────────┤
│      4200 │
└───────────┘

In [9]:
duckdb.read_csv("sample.csv")

┌─────────┬─────────────┬─────────────┬─────────────┐
│ column0 │      a      │      b      │      c      │
│  int64  │   double    │   double    │   double    │
├─────────┼─────────────┼─────────────┼─────────────┤
│       1 │ 0.051017541 │ 0.829733518 │ 0.191106993 │
│       2 │ 0.034442112 │ 0.313852899 │ 0.593982669 │
│       3 │ 0.980742868 │ 0.517989292 │ 0.706687286 │
└─────────┴─────────────┴─────────────┴─────────────┘

In [16]:
import pandas as pd
df = pd.read_csv('sample.csv').iloc[:, 1:]
df

,a,b,c
0,0.051018,0.829734,0.191107
1,0.034442,0.313853,0.593983
2,0.980743,0.517989,0.706687


In [24]:
con = duckdb.connect(database='file.db')
con.sql('CREATE TABLE test (i INTEGER)')
con.sql('INSERT INTO test VALUES (42)')


CatalogException: Catalog Error: Table with name "test" already exists!

In [26]:
con.table('test').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [28]:
con.sql('INSERT INTO test VALUES (44)')


In [29]:
con.table('test').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
│    44 │
└───────┘



In [30]:
with duckdb.connect('test.db') as con:
    con.sql('CREATE TABLE test (i INTEGER)')
    con.sql('INSERT INTO test VALUES (42)')
    con.table('test').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [39]:
with duckdb.connect('test.db') as con:
    for i in range(43, 100):
        con.sql(f'INSERT INTO test VALUES ({i})')
    con.table('test').show()

BinderException: Binder Error: table test has 3 columns but 1 values were supplied

In [38]:
con = duckdb.connect(database='test.db')
result = con.execute('SELECT * FROM test').fetchdf()
print(result)

          a         b         c
0  0.051018  0.829733  0.191107
1  0.034442  0.313853  0.593983
2  0.980743  0.517989  0.706687


In [36]:
# Connect to DuckDB (in-memory database)
con = duckdb.connect(database=':memory:')

con = duckdb.connect(database='test.db')

# Write the DataFrame to a DuckDB table
df.to_sql('test', con, if_exists='replace', index=False)

# Query the table to confirm the write operation
result = con.execute('SELECT * FROM test').fetchdf()
print(result)

# Close the connection
con.close()

          a         b         c
0  0.051018  0.829733  0.191107
1  0.034442  0.313853  0.593983
2  0.980743  0.517989  0.706687


C:\Users\조근하\AppData\Local\Temp\ipykernel_31036\2870466606.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('test', con, if_exists='replace', index=False)


In [21]:
with duckdb.connect(database='scratch01.db') as con:
    # Write the DataFrame to a DuckDB table
    df.to_sql('sf', con, if_exists='replace', index=False)
    # Query the table to confirm the write operation
    result = con.execute('SELECT * FROM sf').fetchdf()
    
print(result)


          a         b         c
0  0.051018  0.829733  0.191107
1  0.034442  0.313853  0.593983
2  0.980743  0.517989  0.706687


C:\Users\조근하\AppData\Local\Temp\ipykernel_31036\1416002065.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('sf', con, if_exists='replace', index=False)
